# Text Classification and Logistic Regression

This activity's learning goals are to work towards being able to...

1. Identify and describe the input and output structure for text classification tasks
2. Explain the differences between generative and discriminative models for classification tasks
3. Define and implement Logistic Regression for classification
4. Practice using PyTorch for doing machine learning

## Brief Review - Classification and Logistic Regression

Recall that text classification is a task where our model must assign a class/label $c \in C$ to some text input $\omega \in \Sigma^*$. Much like with Naive Bayes, we'll be solving this problem by having a lot of labeled training data, pairs $\{(\omega_1, c_1), \dots (\omega_n, c_n)\}$, and using that data to learn the relationship between the data and the labels  (what we call *supervised machine learning*). 

Naive Bayes was an example of *generative* modeling --- we attempted to understand the process that generates the data (or, rather, features of the data) for each class and evaluating the likelihood of the data under each model. Logistic regression forgoes learning anything about the generative process and instead tries to map features of the data directly to class membership

## Our Task Today
Let's build a simple bag-of-words logistic regression model in pytorch! Pytorch is a library for doing machine learning in python that is standard for working with neural networks, and is more than capable to implementing something like logistic regression (which is, in practice, a very simple neural model).

Now, how are we going to implement logistic regression. 

If we are using a bag-of-words model, we're going have features that count each word in our vocabulary. that means we'll have one feature for each word/token in our vocabulary. Since each vocab item can be mapped to some integer, we can store the count of word $i$ (feature $i$, or $f_i$) at index $i$ of a size $\lvert V \rvert$ vector $x$! 

Each of these is multiplied by a coefficient that we'll learn from our training data. We can think of this as another size $\lvert V \rvert$ vector $w$, and then (following Jurafsky and Martin) we can compute a score for our input $w \cdot x$ and then add a learned *bias* term $b$, toss it into a sigmoid function $\sigma$ and then get a probability out. Let's focus (for now) on developing that *forward pass* of our model.

We're going to be discussing a lot of pytorch functions here, so make sure you have a tab with the [official documentation](https://pytorch.org/docs/2.5/) open so you can check the details of each function (and get familiar with navigating the docs!).

### Data and Preprocessing

First, let's figure out our vocab (i.e., *features*). Lets load our data...

In [ ]:
path = "./data/train/{}.txt"

with open(path.format("pos")) as pos_data_f:
    pos_data = pos_data_f.read().lower().split("\n")

with open(path.format("neg")) as neg_data_f:
    neg_data = neg_data_f.read().lower().split("\n")

and have *any* word that appears have it's count as a feature. This is exactly like bag-of-words for naive Bayes, but we'll store the counts as a *vector* (an array, essentially!).

In [ ]:
import torch 

vocab = list(set(" ".join(pos_data + neg_data).split()))
# Construct a map from words to indices into the vocab list
w2idx = {w:i for (i, w) in enumerate(vocab)}

train_data = []

for label, data in enumerate([neg_data, pos_data]):
    for sent in data:
        x = torch.zeros(len(vocab))
        for w in sent.split():
            if w in w2idx:
                x[w2idx[w]] += 1
        train_data.append((x, label))

Let's slow down and see exactly what we're doing. First, for every example, we construct a *tensor* of zeros of size `len(vocab)`. `len(vocab)` is the number of tokens, and thus the number of features! A tensor in pytorch is a generalization of vectors/matrices/etc. A 1D tensor is a vector, a 2D tensor is a matrix, and so on. You can also think of these as N-dimensional arrays!

In [ ]:
print(len(vocab))
print(torch.zeros(len(vocab)))

So we have a vector of 32708 features! 
The dictionary w2idx (word-to-index) is a map from a word to the index into this vector that counts the appearances of that word. The most-nested for-loop iterates through all of the tokens in the example and adds 1 to the corresponding index. We're essentially building a count dictionary, but in a tensor!

So let's look at example 0, the first in `neg_data`. 

In [ ]:
print(neg_data[0].split())

Let's verify we counted the number of appearances of *the* correctly (since neg_data begins train data, the first examples are the same!)

In [ ]:
print(w2idx["the"])

x, l = train_data[0]
print(x[w2idx["the"]])

Verify that this is correct!

### Building our Classfier Class

Now let's design our logistic regression model. We do this in pytorch designing a class that inherits from `nn.module`, a parent class for all pytorch models. Our focus is on initalizing our model and writing a `forward` method that defines a forward pass of our model (i.e., how we would classify an example with a trained model!)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class LogisticSentimentClassifier(nn.Module):
    def __init__(self, vocab):
        # Call the nn.Module constructor
        super(LogisticSentimentClassifier, self).__init__()

        # Run our constructor
        self.vocab = vocab

        # Wrapping in nn.Parameter indicates that these are our
        # model's learned parameters!
        self.w = nn.Parameter(torch.empty(len(vocab)))
        self.b = nn.Parameter(torch.empty(1))

        # Randomly initialize the parameters Don't worry about this
        nn.init.uniform_(self.w, -1.0/np.sqrt(len(self.vocab)), 1.0/np.sqrt(len(self.vocab)))

    def forward(self, x):
        logit = torch.dot(self.w, x) + self.b
        return F.sigmoid(logit).view(-1)

Now we could stop here, but if we're clever we can design our classifier so it can classify multiple things at once (through the power of matrix multiplication!). 

To do this, we need to think of our weight $w$ as a $\lvert V \rvert$ x 1 matrix and our input $X$ as a $N$ x $\lvert V \rvert$ (where $N$ is the number of inputs). That way doing an $X \cdot w$ multiplication gives us an $N$ x 1 matrix (i.e., a vector with the predictions for all $N$ of the inputs!)

In [ ]:
class LogisticSentimentClassifierBatched(LogisticSentimentClassifier):
    def __init__(self, vocab):
        # Call the nn.Module constructor
        super(LogisticSentimentClassifier, self).__init__()

        # Run our constructor
        self.vocab = vocab

        # Wrapping in nn.Parameter indicates that these are our
        # model's learned parameters!
        self.w = nn.Parameter(torch.empty(len(vocab), 1))
        self.b = nn.Parameter(torch.empty(1))

        # Randomly initialize the parameters
        nn.init.uniform_(self.w, -1.0/np.sqrt(len(self.vocab)), 1.0/np.sqrt(len(self.vocab)))

    
    def forward(self, x):
        return F.sigmoid(torch.matmul(x,self.w) + self.b).view(-1)

### Training a model

Now we can build a `LogisticSentimentClassifier` or `logisticSentimentClassifierBatched` object, but it will be a bad classifier, because our parameters are all random! We need to learn the *right* values for each parameter! We do this through an optimization process:

1. Make a prediction (or predictions) with a bad model
2. Compute a measure of how poorly the model predicted (called a *loss function*)
3. Determine how our parameters must change to make those predictions better (called a *backward pass*, or often *backpropagation*, the algorithm that efficiently computes this for large models)
4. Adjust our parameters a small amount in a direction that would make those predictions better

If you take a proper Machine Learning class, you'll learn all about the classic optimization method in this setting, Stochastic Gradient Descent. J&M has a solid introduction! For us, we can focus on using it in PyTorch!

In [ ]:
import torch.optim as optim
import random

logisticSentiment = LogisticSentimentClassifierBatched(vocab)

We build an `Optimizer` object that will manage updating our model's parameters --- the coefficients of the regression!

To do this, we need to specify a *learning rate*. This tells the model how fast we should change parameters to better fit our data. To high of a learning rate, and we'll overshoot (and might even get `nan`s as parameter values as we overflow!). Too small, and it will take a LOT of training examples to learn good values of our model.

In [ ]:
learning_rate = 0.001 

optimizer = optim.SGD(logisticSentiment.parameters(), lr=learning_rate)

We also need to define a *loss function*. This tells us what to minimize in order to improve. Here, we want to use *negative log likelihood*.

**Scan back to your reading to find the formula for NLL** (fun fact: an approximation of the *cross-entropy*) and write a method to compute it given the true label $y$ and the predicted probability of a positive label $\hat{y}$.

In [ ]:
def nll(y, y_hat):
    # TODO: Implement NLL

We can also take a look at the *parameters* of our model! These should be our coefficients $w$ and the bias term $b$.

In [ ]:
for param in logisticSentiment.parameters():
    print(param)

Now we prep our training data! We could go example by example, but a problem is that this makes training a little *unstable*. When we nudge our parameter values for each example, they can jump around a lot to try to adjust to individual examples! 

Instead, we might want to adjust in **batches** based on the *average* loss. That way we optimize so that we predict the entire batch of examples better at each step! 

Since we build a classifier that can classify multiple things at once, the only thing to adjust is our data!

If I want to put $N$ different $\lvert V \rvert$ training examples together into an $N$ x $\lvert V \rvert$ matrix, we can use `torch.stack`

In [ ]:
print([x for x, y in train_data[:3]])

print(torch.stack([x for x, y in train_data[:3]]))

If that's hard to read, we can always look at the *size* of a tensor

In [ ]:
print(torch.stack([x for x, y in train_data[:3]]).size())

Now we have a batch of 3 examples! We can also do the same for the corresponding labels, but since they aren't yet a tensor, we can call torch.tensor to turn a list of them into one!

In [ ]:
torch.tensor([y for x,y in train_data[:3]])

Let's turn our training data into a list of batches of size at-most `batch_size`, with each batch being a pair of an $N$ x $\lvert V \rvert$ input matrix X and an $N$-length vector of the correct labels for each of the $N$ examples. 

In [ ]:
from typing import Iterable, Tuple

def batch(data : Iterable[Tuple[torch.Tensor, int]], batch_size : int) -> Iterable[Tuple[torch.Tensor, torch.Tensor]]:
    # TODO
    return []

batch_size = 20 # A parameter you can change!

batched_train_data = batch(train_data, batch_size)

Now finally to our 4 step loop! 

For every pair of X (input) and Y (label), we predict, measure our loss, compute whether we should increase or decrease our parameters with `loss.backward()`, and then update our parameters accordingly with `optimizer.step()`. One small technical detail is that we need to begin or end each iteration with calls to `optimizer.zero_grad()`, which resets some internal information about how the parameters should be updated (their *gradients*, named such for calculus reasons!). 

In [ ]:
for X, Y in batched_train_data:
    
    # reset for the next iteration
    optimizer.zero_grad()

    # predict
    Y_hat = logisticSentiment(X)

    loss = 0
    
    # how bad did we predict each thing in our batch
    # This can be optimized further with an NLL function that works over tensors!
    # do
    for y, y_hat in zip(Y, Y_hat):
        loss += nll(y, y_hat) 

    loss /= len(ys)
    
    # how should parameters be adjusted
    loss.backward()

    # adjust our parameters
    optimizer.step()

This should get us a somewhat better model! Let's see how good it is!

Complete this implementation of an evaluation function that computes the accuracy of the classifier. You'll need to determine how to pick the label (positive/1 or negative/0) based on the probability the model gives out. This is where we use our *decision boundary* -- **reference the reading**!

In [ ]:
import matplotlib 

def eval(model, data):
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in data:
            y_hat = None  # TODO: Fix
            if y_hat == y:
                correct += 1
            total += 1

    return correct/total

eval(logisticSentiment, train_data)

Initialization is random, so your model might be good? But chances are it's not too far from *chance* accuracy (~50\%) for a binary classifier.

In [ ]:
test_path = "./data/test/{}.txt"

with open(test_path.format("pos")) as pos_data_f:
    pos_test = pos_data_f.read().split("\n")

with open(test_path.format("neg")) as neg_data_f:
    neg_test = neg_data_f.read().split("\n")

test_data = []
for label, data in enumerate([neg_test, pos_test]):
    for sent in data:
        x = torch.zeros(len(vocab))
        for w in sent.split():
            if w in w2idx:
                x[w2idx[w]] += 1
        test_data.append((x, label))

eval(logisticSentiment, test_data)

I unfortunately cannot predict randomness here as well, but you're probably not doing too great? What should make the model better is more training time though! 

#### Training over Multiple Epochs

We have only seen each example once so far, which is fine for something like Naive Bayes. However, when optimizing, we're in a constant tug of war with our parameters, and if our learning rate is reasonable we can likely do better by seeing the same data again. 

Each loop over the dataset during training is called an *epoch*, and we often like to have a bunch of them! Here, I'll ask you to *experiment*, but in practice there are a few strategies for deciding how many epochs to have

1. There is a *practical* upper bound --- compute time, patience, etc.
2. We can try and *converge*: We're trying to minimize the log-likelihood, so we can keep going until we have run into that upper bound or we stop improving.
3. We don't have to use the log likelihood as a measure to stop early. In fact, this can be problematic if this causes us to overfit on our training data (think about MLE n-gram models!). We can also implement early stopping based on whether or not log-likelihood over a *validation set* stops improving.
4. There are even fancier strategies that, instead of stopping, modify the learning rate with the intuition that a smaller learning rate can help us continue to improve our model! *Learning rate scheduling* is fancy, but often useful for getting models that work just a little bit better!

That aside, here I give you a template for going over 100 epochs (not bad for our tiny corpus). Once you fill in the blanks inside of the loop, start playing around with different numbers. Do you overfit? Do you converge within time? Does loss correlate neatly with accuracy? 

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    random.shuffle(train_data)
    total_loss = 0
    batched_train_data = batch(train_data, batch_size)

    for X, Y in batched_train_data:    
        # reset for the next iteration
        optimizer.zero_grad()
    
        # predict
        Y_hat = logisticSentiment(X)
    
        loss = 0
        
        # how bad did we predict (batched!)
        for y, y_hat in zip(Y, Y_hat):
            loss += nll(y, y_hat) 
    
        loss /= len(ys)
        total_loss += loss.item()
        
        # how should parameters be adjusted
        loss.backward()
    
        # adjust our parameters
        optimizer.step()

    # Log every 5 epochs!
    if epoch % 5 == 0:
        print("epoch {} | train loss: {:.5} | train acc: {:.3} | test acc: {:.3}".format(epoch, 
                                                                                         total_loss/batch_count, 
                                                                                         eval(logisticSentiment, train_data), 
                                                                                         eval(logisticSentiment, test_data)))

#### Bonus: Everything's a batch!

Since we're using a small model like logistic regression and our dataset is also very small, we could actually go extreme with batching and made parameter updates based on the predictions of our model over the entire corpus. Try that out: You won't need an inner loop at all!

In [ ]:
# reinitialize so we're not starting with a trained model
logisticSentiment = LogisticSentimentClassifierBatched(vocab)

for epoch in range(1,301):
    random.shuffle(train_data)

    X = None # TODO!
    Y = None # TODO!
    
    # reset for the next iteration
    optimizer.zero_grad()
    
    # predict
    Y_hat = logisticSentiment(X)
    
    loss = 0
    
    # how bad did we predict (over the full training set!)
    for y, y_hat in zip(Y, Y_hat):
        loss += nll(y, y_hat) 
    
    loss /= len(ys)
    
    # how should parameters be adjusted
    loss.backward()

    # adjust our parameters
    optimizer.step()
